In [180]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
import os

In [181]:
source_path="/content/train-source.txt"
target_path="/content/train-target.txt"

In [222]:
def data_preprocessing(file):
  data_file= []
  with open(file,'r',encoding='utf-8') as f:
    data = f.read()
    data = data.strip().split('</s>')
    for i in data:
      data = re.sub("\n.\n',",'.',i)
      data = re.sub('\n<s>.','',data)
      data = re.sub('\n<s>','',data)
      data = re.sub('\n',' ',data)
      data = re.sub('<s>','',data)
      data_file.append(data)
  return data_file

In [223]:
source_file=data_preprocessing(source_path)
target_file=data_preprocessing(target_path)

In [184]:
df1 = pd.DataFrame({'Source': source_file})
df1

,Source
0,"Cinnte go leór , thiocfadh dóbhtha bás a fhagh..."
1,( Bhí sé follasach go rabh an poll sin ag fosc...
2,) D'fhéadfadh siad bás ' fhagháil ar a bhruach...
3,Thiocfadh dóbhtha fosta lámh a chur ina mbás f...
4,"' Na dhiaidh sin , bhí rud éigin do-chreidte a..."
...,...
45167,Is fearr liom tarraingt ortha ná áit ar bith e...
45168,I dtoigh Johnny Shéimisín is mó ghním a ' t-ái...
45169,Creidim nach mbíonn mo ruaig síos anois comh m...
45170,"Mar sin de , caithfidh mé suidhe sa teach ."


In [185]:
df2 = pd.DataFrame({'Target': target_file})
df2

,Target
0,"Cinnte go leor , thiocfadh dóibh bás a fháil a..."
1,Bhí sé follasach go raibh an poll sin ag foscl...
2,D'fhéadfadh siad bás a fháil ar a bhruach agus...
3,Thiocfadh dóibh fosta lámh a chur ina mbás féi...
4,Ina dhiaidh sin bhí rud éigin dochreidte agus ...
...,...
45167,Is fearr liom tarraingt orthu ná áit ar bith e...
45168,Tigh Johnny Shéamaisín is mó a ním an t-airneá...
45169,Creidim nach mbíonn mo ruaig síos anois chomh ...
45170,"Mar sin de , caithfidh mé suí sa teach ."


In [186]:
df_merged = pd.concat([df1, df2], axis=1)
df_merged

,Source,Target
0,"Cinnte go leór , thiocfadh dóbhtha bás a fhagh...","Cinnte go leor , thiocfadh dóibh bás a fháil a..."
1,( Bhí sé follasach go rabh an poll sin ag fosc...,Bhí sé follasach go raibh an poll sin ag foscl...
2,) D'fhéadfadh siad bás ' fhagháil ar a bhruach...,D'fhéadfadh siad bás a fháil ar a bhruach agus...
3,Thiocfadh dóbhtha fosta lámh a chur ina mbás f...,Thiocfadh dóibh fosta lámh a chur ina mbás féi...
4,"' Na dhiaidh sin , bhí rud éigin do-chreidte a...",Ina dhiaidh sin bhí rud éigin dochreidte agus ...
...,...,...
45167,Is fearr liom tarraingt ortha ná áit ar bith e...,Is fearr liom tarraingt orthu ná áit ar bith e...
45168,I dtoigh Johnny Shéimisín is mó ghním a ' t-ái...,Tigh Johnny Shéamaisín is mó a ním an t-airneá...
45169,Creidim nach mbíonn mo ruaig síos anois comh m...,Creidim nach mbíonn mo ruaig síos anois chomh ...
45170,"Mar sin de , caithfidh mé suidhe sa teach .","Mar sin de , caithfidh mé suí sa teach ."


In [187]:
df_merged['Source']=df_merged['Source'].apply(lambda x: x.lower())
df_merged['Target']=df_merged['Target'].apply(lambda x: x.lower())

In [188]:
df_merged

,Source,Target
0,"cinnte go leór , thiocfadh dóbhtha bás a fhagh...","cinnte go leor , thiocfadh dóibh bás a fháil a..."
1,( bhí sé follasach go rabh an poll sin ag fosc...,bhí sé follasach go raibh an poll sin ag foscl...
2,) d'fhéadfadh siad bás ' fhagháil ar a bhruach...,d'fhéadfadh siad bás a fháil ar a bhruach agus...
3,thiocfadh dóbhtha fosta lámh a chur ina mbás f...,thiocfadh dóibh fosta lámh a chur ina mbás féi...
4,"' na dhiaidh sin , bhí rud éigin do-chreidte a...",ina dhiaidh sin bhí rud éigin dochreidte agus ...
...,...,...
45167,is fearr liom tarraingt ortha ná áit ar bith e...,is fearr liom tarraingt orthu ná áit ar bith e...
45168,i dtoigh johnny shéimisín is mó ghním a ' t-ái...,tigh johnny shéamaisín is mó a ním an t-airneá...
45169,creidim nach mbíonn mo ruaig síos anois comh m...,creidim nach mbíonn mo ruaig síos anois chomh ...
45170,"mar sin de , caithfidh mé suidhe sa teach .","mar sin de , caithfidh mé suí sa teach ."


In [189]:
df_merged['Source']=df_merged['Source'].apply(lambda x: re.sub("'", '', x))
df_merged['Target']=df_merged['Target'].apply(lambda x: re.sub("'", '', x))

In [190]:
df_merged.isnull().sum()

Source    0
Target    0
dtype: int64

In [191]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
df_merged['Source']=df_merged['Source'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
df_merged['Target']=df_merged['Target'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [192]:
df_merged['Target'] = df_merged['Target'].apply(lambda x : 'START_ '+ x + ' _END')

In [193]:
df_merged.head()

,Source,Target
0,cinnte go leór thiocfadh dóbhtha bás a fhaghá...,START_ cinnte go leor thiocfadh dóibh bás a f...
1,bhí sé follasach go rabh an poll sin ag foscl...,START_ bhí sé follasach go raibh an poll sin a...
2,dfhéadfadh siad bás fhagháil ar a bhruach ag...,START_ dfhéadfadh siad bás a fháil ar a bhruac...
3,thiocfadh dóbhtha fosta lámh a chur ina mbás f...,START_ thiocfadh dóibh fosta lámh a chur ina m...
4,na dhiaidh sin bhí rud éigin dochreidte agus...,START_ ina dhiaidh sin bhí rud éigin dochreidt...


In [194]:
all_source_words=set()
for i in df_merged['Source']:
    for word in i.split():
        if word not in all_source_words:
            all_source_words.add(word)

In [195]:
all_target_words=set()
for j in df_merged['Target']:
    for word in j.split():
        if word not in all_target_words:
            all_target_words.add(word)

In [196]:
len(all_source_words)

29467

In [197]:
len(all_target_words)

25073

In [198]:
df_merged['Length of Source']=df_merged['Source'].apply(lambda x:len(x.split(" ")))
df_merged['Length of Target']=df_merged['Target'].apply(lambda x:len(x.split(" ")))

In [199]:
df_merged=df_merged[df_merged['Length of Source']<=30]
df_merged=df_merged[df_merged['Length of Target']<=30]

In [200]:
print("maximum length of Source Sentence ",max(df_merged['Length of Source']))
print("maximum length of Target Sentence ",max(df_merged['Length of Target']))

maximum length of Source Sentence  30
maximum length of Target Sentence  30


In [201]:
df_merged.shape

(38581, 4)

In [203]:
max_length_src=max(df_merged['Length of Source'])
max_length_tar=max(df_merged['Length of Target'])

In [204]:
source_words = sorted(list(all_source_words))
target_words = sorted(list(all_target_words))
encoder_tokens = len(all_source_words)
decoder_tokens = len(all_target_words)
encoder_tokens, decoder_tokens

(29467, 25073)

In [205]:
decoder_tokens += 1 # For zero padding
decoder_tokens

25074

In [206]:
source_token_index = dict([(word, i+1) for i, word in enumerate(source_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [207]:
reverse_source_char_index = dict((i, word) for word, i in source_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [208]:
df_merged.head(10)

,Source,Target,Length of Source,Length of Target
0,cinnte go leór thiocfadh dóbhtha bás a fhaghá...,START_ cinnte go leor thiocfadh dóibh bás a f...,15,17
1,bhí sé follasach go rabh an poll sin ag foscl...,START_ bhí sé follasach go raibh an poll sin a...,26,27
2,dfhéadfadh siad bás fhagháil ar a bhruach ag...,START_ dfhéadfadh siad bás a fháil ar a bhruac...,22,24
4,na dhiaidh sin bhí rud éigin dochreidte agus...,START_ ina dhiaidh sin bhí rud éigin dochreidt...,17,17
5,nuair a scríobh siad an litir sin bhí siad ara...,START_ nuair a scríobh siad an litir sin bhí s...,18,20
7,bfhuras a fheiceáil cá dtáinig an chaoirfheóil,START_ bfhurasta a fheiceáil cá dtáinig an cha...,8,10
8,ar urlár na huaimhe bhí cnámha caorach ar gear...,START_ ar urlár na huaimhe bhí cnámha caorach ...,14,16
9,séard a béigean don chaoraigh a ghabháil le be...,START_ is éard ab éigean don chaora a ghabháil...,18,21
10,níor bhac muid leis an taiscidh sin,START_ níor bhac muid leis an taisce sin _END,8,10
11,go deimhin is mó a mhéaduigh muid í ná dfhága...,START_ go deimhin is mó a mhéadaigh muid í n...,24,28


In [209]:
# Train - Test Split
X = df_merged.Source
y = df_merged.Target

In [210]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape

((34722,), (3859,))

In [211]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = source_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [212]:
latent_dim = 50

In [213]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [214]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [215]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [216]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 10

In [217]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, None, 50)     1473350     ['input_3[0][0]']                
                                                                                                  
 embedding_3 (Embedding)        (None, None, 50)     1253700     ['input_4[0][0]']                
                                                                                            

In [224]:
model.fit(generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs) 1\\\

Epoch 1/10


InvalidArgumentError: ignored